In [ ]:
from functools import partial
import os
import tempfile
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray import train
from ray.train import Checkpoint, get_checkpoint
from ray.tune.schedulers import ASHAScheduler
import ray.cloudpickle as pickle
from ultralytics import YOLO

In [ ]:
#best_model = YOLO("best_model.pt")

#print(f"The task type of the model is: {best_model.task}")

In [ ]:
TASK2METRIC = {
    "detect": "metrics/mAP50(B)", #detect 지표를 mAP50-95에서 50으로 변경하기 위함
    "segment": "metrics/mAP50-95(M)",
    "classify": "metrics/accuracy_top1",
    "pose": "metrics/mAP50-95(P)",
    "obb": "metrics/mAP50-95(B)",
}

In [ ]:
experiment_name = 'tune'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

# Load a YOLOv8n model
best_model_tuning = YOLO("yolov8n.pt")

task = best_model_tuning.task
metric = TASK2METRIC[task] 

result_grid = best_model_tuning.tune(
    data="/home/under1/Detect/jeongui/sagittal/sagittal_t1wce_2_class.yaml",
    epochs=200,
    use_ray=True,
    name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'),
    gpu_per_trial=1  # 각 트라이얼에 1개의 GPU 할당
)

In [ ]:
for i, result in enumerate(result_grid):
    print(f"Trial #{i}: Configuration: {result.config}, Last Reported Metrics: {result.metrics}")

In [ ]:
# Extract the best trial configuration
best_result = result_grid.get_best_result(metric=metric, mode="max")
best_config = best_result.config
print("Best hyperparameters found were: ", best_config)

In [ ]:
experiment_name = 'train'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

# 'data' 항목 제거
if 'data' in best_config:
    del best_config['data']
if 'name' in best_config:
    del best_config['name']

# 최고의 하이퍼파라미터로 모델 재훈련
best_model = YOLO("yolov8n.pt")  # 모델 재초기화
best_results = best_model.train(data="sagittal_t1wce_2_class.yaml",
                                optimizer='SGD',
                                augment=True,
                                name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'),
                                **best_config)

In [ ]:
best_results.results_dict

In [ ]:
experiment_name = 'val'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

best_model = YOLO("/home/under1/Detect/jeongui/sagittal/runs/detect/train/train_raytune/weights/best.pt")
best_model.val(name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'))

In [ ]:
# test

experiment_name = 'test'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

best_model.val(name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'), split="test")